In [ ]:
%matplotlib inline

import os
from os.path import join as pjoin

import numpy as np
import pandas as pd
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

import seaborn as sns

sns.set_context('paper')
sns.set_style('whitegrid')

In [ ]:
exp_inputs = ['2020', '2020_LUO']
region = 'MB_CODE'

datapath = "C:/WorkSpace/swhaq/data/impact"
scenario_list = pd.read_csv("C:/WorkSpace/swhaq/configuration/scenario_lookup_table.csv")
bins = [0.0, 0.02, 0.1, 0.2, 0.5, 1.0]
labels = ['Negligible', 'Slight', 'Moderate', 'Extensive', 'Complete']


def categorise(df, category, bins, labels):
    df[category] = pd.cut(df['structural_loss_ratio'], bins=bins, labels=labels, right=False)
    return df

In [ ]:
statdf = pd.DataFrame(columns=['scenario', 'source', 'mdi', 'median', 'mean'])

for s in scenario_list['ID']:
    print(s)
    basedf = pd.read_csv(pjoin(datapath, "NEXISV10", s, f'QFES_{s}.csv'))
    basedf = categorise(basedf, 'damage_state', bins, labels)
    df2020 = pd.read_csv(pjoin(datapath, "2020", s, f'QFES_{s}.csv'))
    df2020 = categorise(df2020, 'damage_state', bins, labels)
    df2020LUO = pd.read_csv(pjoin(datapath, "2020_LUO", s, f'QFES_{s}.csv'))
    df2020LUO = categorise(df2020LUO, 'damage_state', bins, labels)

    basedfcount = basedf.groupby([region, 'damage_state']).size().unstack(fill_value=0)
    df2020count = df2020.groupby([region, 'damage_state']).size().unstack(fill_value=0)
    df2020LUOcount = df2020LUO.groupby([region, 'damage_state']).size().unstack(fill_value=0)

    df2020_change = basedfcount.subtract(df2020count)
    df2020LUO_change = basedfcount.subtract(df2020LUOcount)
    
    df2020_change.to_csv(pjoin(datapath, f'dschange.{s}.2020.{region}.csv'))
    df2020LUO_change.to_csv(pjoin(datapath, f'dschange.{s}.2020LUO.{region}.csv'))
    
    basedfmdi = basedf.groupby(region)['structural_loss_ratio'].mean()
    df2020mdi = df2020.groupby(region)['structural_loss_ratio'].mean()
    df2020LUOmdi = df2020LUO.groupby(region)['structural_loss_ratio'].mean()

    df2020mdi.subtract(basedfmdi).to_csv(pjoin(datapath, f'mdichange.{s}.2020.{region}.csv'))
    df2020LUOmdi.subtract(basedfmdi).to_csv(pjoin(datapath, f'mdichange.{s}.2020LUO.{region}.csv'))
    
    fig, ax = plt.subplots(2,1, sharex=True, figsize=(8,6))
    df2020mdidelta = df2020mdi.subtract(basedfmdi).dropna()
    df2020LUOmdidelta = df2020LUOmdi.subtract(basedfmdi).dropna()
    sns.distplot(df2020mdidelta.values, ax=ax[0], kde=True)
    sns.distplot(df2020LUOmdidelta.values, ax=ax[1], kde=True)
    ax[0].axvline(0, linestyle='--', color='0.5')
    ax[0].axvline(df2020mdidelta.median(), linestyle='--', )
    ax[1].axvline(0, linestyle='--', color='0.5')
    ax[1].axvline(df2020LUOmdidelta.median(), linestyle='--', )
    ax[0].set_title(f"Full NEXIS update (n={df2020mdidelta.count()})")
    ax[1].set_title("Land-Use only")
    plt.savefig(pjoin(datapath, f'mdichange.{s}.{region}.png'), bbox_inches='tight')
    plt.close(fig)
    
    statdf = statdf.append({'scenario':s, 'source':'2020',
                            'mdi':df2020mdi.dropna().mean(),
                            'median': df2020mdidelta.median(),
                            'mean': df2020mdidelta.mean()}, ignore_index=True)
    statdf = statdf.append({'scenario':s, 'source':'2020LUO',
                            'mdi':df2020LUOmdi.dropna().mean(),
                            'median': df2020LUOmdidelta.median(),
                            'mean': df2020LUOmdidelta.mean()}, ignore_index=True)

In [ ]:
statdf.to_csv(pjoin(datapath, "damage_state_change_statistics.csv"))

In [ ]:
s = '006-05866'

fig, ax = plt.subplots(1, 1, subplot_kw={'projection':ccrs.PlateCarree()}, figsize=(8, 8))
df2020 = gpd.read_file(pjoin(datapath, "2020", s, f'QFES_{s}.shp'))
df2018 = gpd.read_file(pjoin(datapath, "NEXISV10", s, f'QFES_{s}.shp'))

df2020.plot(column='slr_mean', ax=ax, legend=True, edgecolor='gray', linewidth=0.25,
            legend_kwds={"label":"Mean SLR"}, cmap='viridis_r')

c = cfeature.GSHHSFeature(scale='full')
ax.add_feature(c)
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='10m',
        facecolor='none')
ax.add_feature(states_provinces, edgecolor='0.5', linestyle='--')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, )
gl.xlabels_top = False
gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER


In [ ]:
cmap = sns.diverging_palette(240, 10, n=9, as_cmap=True)
c = cfeature.GSHHSFeature(scale='full')
states_provinces = cfeature.NaturalEarthFeature(
            category='cultural',
            name='admin_1_states_provinces_lines',
            scale='10m',
            facecolor='none')

for s in scenario_list['ID']:
    df2020 = gpd.read_file(pjoin(datapath, "2020", s, f'QFES_{s}.shp'))
    df2018 = gpd.read_file(pjoin(datapath, "NEXISV10", s, f'QFES_{s}.shp'))
    delta = df2020.copy()
    delta['slr_mean'] = df2020['slr_mean'].subtract(df2018['slr_mean'])
    fig, ax = plt.subplots(1, 1, subplot_kw={'projection':ccrs.PlateCarree()}, figsize=(12, 12))
    delta.plot(column='slr_mean', ax=ax, legend=True, edgecolor='gray', linewidth=0.25,
               cmap=cmap,
               scheme="user_defined",classification_kwds={"bins":np.arange(-1, 1.1, 0.1)})
    
    ax.add_feature(c)
    ax.add_feature(states_provinces, edgecolor='0.5', linestyle='--')
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    delta.to_file(pjoin(datapath, "2020", s, f"QFES_{s}.difference.shp"))
    plt.savefig(pjoin(datapath, "2020", s, f"QFES_{s}.difference.png"), bbox_inches="tight", dpi=600)